## Trabalho De Física - Projeto n° 3

### Por: Rodrigo Freitas, Fernando Moraes, Davi Araújo

### Para uma experiência interativa sem o código utilize a ferramenta nativa do ecosistema jupyter, o voila.

Para utilização execute o programa e interaja com os sliders ou células para dar a entrada aos dados que serão utilizados para a simulação de um movimento causado por uma força sobre um corpo massivo.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# --- Pegar o que é preciso para os graficos ---
def computar_variaveis_E_resultados(m, F, tmax, n=400):
    t = np.linspace(0, tmax, n)
    a = F / float(m)
    x = 0.5 * a * t**2
    v = a * t
    return t, x, v, a

# --- Bota os Widgets em disposicao ---
massa_slider = widgets.IntSlider(
    value=5, min=1, max=200, step=1, description='Massa (kg):',
    continuous_update=True, layout=widgets.Layout(width='420px')
)
força_slider = widgets.IntSlider(
    value=10, min=-500, max=500, step=1, description='Força (N):',
    continuous_update=True, layout=widgets.Layout(width='420px')
)
tmax_slider = widgets.FloatSlider(
    value=10.0, min=0.5, max=60.0, step=0.5, description='Tempo max (s):',
    continuous_update=True, layout=widgets.Layout(width='420px')
)

massa_display = widgets.FloatText(value=massa_slider.value, description='m (kg):',
                                 layout=widgets.Layout(width='160px'))
força_display = widgets.FloatText(value=força_slider.value, description='F (N):',
                                  layout=widgets.Layout(width='160px'))

# Conecta os valores dos sliders com os displays
widgets.jslink((massa_slider, 'value'), (massa_display, 'value'))
widgets.jslink((força_slider, 'value'), (força_display, 'value'))

# --- (Implementação da funão /// pega os valores de cada slide e vai jogando nos objetos k0) ---
t0, x0, v0, a0 = computar_variaveis_E_resultados(massa_slider.value, força_slider.value, tmax_slider.value)

posiçao_cor = 'rgba(0,102,204,1)'       # POCIÇÃO (azul)
velocidade_cor = 'rgba(160,32,240,0.95)'   # VELOCIDADE (roxa)

# ---!!!! AQUI EH O GRAFICO!!!!---
fig = go.FigureWidget(
    data=[
        # {LINHA DA POSIÇÃO}
        go.Scatter(
            x=t0, y=x0, mode='lines', name='Posição x(t)',
            line=dict(width=2, color=posiçao_cor),
            hovertemplate='t=%{x:.2f}s<br>x=%{y:.3f} m'
        ),
        # {LINHA DA VELOCIDADE}
        go.Scatter(
            x=t0, y=v0, mode='lines', name='Velocidade v(t)',
            line=dict(width=2, dash='dash', color=velocidade_cor),
            hovertemplate='t=%{x:.2f}s<br>v=%{y:.3f} m/s',
            opacity=0.95
        ),
        # MARCADOR DA{POSIÇÃO FINAL}
        go.Scatter(
            x=[t0[-1]], y=[x0[-1]], mode='markers',
            marker=dict(size=9, color=posiçao_cor, line=dict(width=1, color='white')),
            name='pos final', showlegend=False, hoverinfo='skip'
        ),
        # MARCADOR DA {VELOCIDADE FINAL}
        go.Scatter(
            x=[t0[-1]], y=[v0[-1]], mode='markers',
            marker=dict(size=9, color=velocidade_cor, symbol='diamond', line=dict(width=1, color='white')),
            name='vel final', showlegend=False, hoverinfo='skip'
        )
    ],
    layout=go.Layout(
        title=f'Posição vs. Tempo — m={massa_slider.value} kg, F={força_slider.value} N, a={a0:.3f} m/s²',
        xaxis_title='Tempo (s)',
        yaxis_title='Posição (m) / Velocidade (m/s)',
        template='simple_white',
        plot_bgcolor='rgba(0,0,0,0)',
        height=480,
        width=860,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
)

# (TEMPLATE DE LAYOUT PRA LIVRARIA)
x_end = x0[-1]
v_end = v0[-1]
t_end = t0[-1]
ann = dict(
    x=t_end, y=x_end,
    xref='x', yref='y',
    text=f'x = {x_end:.3f} m<br>v = {v_end:.3f} m/s',
    showarrow=True,
    arrowhead=2,
    ax=-50,
    ay=-50,
    bgcolor='rgba(255,255,255,0.9)',
    bordercolor='black',
    font=dict(size=11)
)
fig.layout.annotations = [ann]

# [CONFIG PRA DEIXAR MAIS BUNITIN] remove grid pra ficar minimalistinha
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# --- (OS OBSERVERS CHAMAN A FUNCAO PRA DAR UPDATE EM TDS AS PARADA) ---
def _update_plot(change=None):
    m = massa_slider.value
    F = força_slider.value
    tmax = tmax_slider.value
    t, x, v, a = computar_variaveis_E_resultados(m, F, tmax)
    x_end = x[-1]
    v_end = v[-1]
    t_end = t[-1]
    annotation_text = f'x = {x_end:.3f} m<br>v = {v_end:.3f} m/s'
    # (offset CONFERIR PRA VER SE N DA ERRO DEPENDENDO)
    ax = -60 if x_end >= 0 else 60
    ay = -60

    with fig.batch_update():
        # (atualiza as linha/traces) 
        fig.data[0].x = t
        fig.data[0].y = x
        fig.data[1].x = t
        fig.data[1].y = v
        # (atualiza as info no final do tempo 'tmax')
        fig.data[2].x = [t_end]
        fig.data[2].y = [x_end]
        fig.data[3].x = [t_end]
        fig.data[3].y = [v_end]
        # (atualiza os anotados)
        fig.layout.annotations = [dict(
            x=t_end, y=x_end, xref='x', yref='y',
            text=annotation_text,
            showarrow=True, arrowhead=2, ax=ax, ay=ay,
            bgcolor='rgba(255,255,255,0.95)',
            bordercolor='black',
            font=dict(size=11)
        )]
        # (MUDA O TITULO)
        fig.layout.title = f'Posição vs. Tempo — m={m} kg, F={F} N, a={a:.3f} m/s²'

# CONECTA OS LISTENERS/OBSERVERS PRA QUANDO INTERAGIR -> REAGIR (Usa o .observe em todos os slider pra toda vez que mudar os kra chamar a funçao de update no grafico)
massa_slider.observe(_update_plot, names='value')
força_slider.observe(_update_plot, names='value')
tmax_slider.observe(_update_plot, names='value')

# (plotagem)
_update_plot()

# CONFIG de layout & display
controls = widgets.VBox([massa_slider, força_slider, tmax_slider])
displays = widgets.VBox([massa_display, força_display])
ui = widgets.VBox([widgets.HBox([controls, displays]), fig])
display(ui)